In this notebook we'll explore topic modeling to discover broad themes in a collection of movie summaries.

In [7]:
import nltk
import re
import gensim
from gensim import corpora
import operator

nltk.download('stopwords')
from nltk.corpus import stopwords

import numpy as np
import random

random.seed(1)
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/piadeshpande/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/piadeshpande/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
def read_stopwords(filename):
    stopwords={}
    with open(filename) as file:
        for line in file:
            stopwords[line.rstrip()]=1
    return stopwords

Since we're running topic modeling on texts with lots of names, we'll add the Jockers list of stopwords (which includes character names) to our stoplist.

In [3]:
stop_words = {k:1 for k in stopwords.words('english')}
stop_words.update(read_stopwords("../data/jockers.stopwords"))
stop_words["'s"]=1
stop_words=list(stop_words.keys())

In [4]:
def filter(word, stopwords):
    
    """ Function to exclude words from a text """
    
    # no stopwords
    if word in stopwords:
        return False
    
    # has to contain at least one letter
    if re.search("[A-Za-z]", word) is not None:
        return True
    
    return False

In [5]:
def read_docs(plotFile, metadataFile, stopwords):
    
    names={}
    box={}
    
    with open(metadataFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            name=cols[2]
            boxoffice=cols[4]
            if len(boxoffice) != 0:
                box[idd]=int(boxoffice)
                names[idd]=name
    
    n=5000
    target_movies={}


    sorted_box = sorted(box.items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_box[:n]:
        target_movies[k]=names[k]
    
    docs=[]
    names=[]
   
    with open(plotFile, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            idd=cols[0]
            text=cols[1]
            
            if idd in target_movies:
                tokens=nltk.word_tokenize(text.lower())
                tokens=[x for x in tokens if filter(x, stopwords)]
                docs.append(tokens)
                name=target_movies[idd]
                names.append(name)
    return docs, names

We'll read in summaries of the 5,000 movies with the highest box office revenues.

In [8]:
metadataFile="../data/movie.metadata.tsv"
plotFile="../data/plot_summaries.txt"
data, doc_names=read_docs(plotFile, metadataFile, stop_words)

We will convert the movie summaries into a bag-of-words representation using gensim's [corpora.dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) methods.

In [9]:
# Create vocab from data; restrict vocab to only the top 10K terms that show up in at least 5 documents 
# and no more than 50% of all documents

dictionary = corpora.Dictionary(data)
dictionary.filter_extremes(no_below=5, no_above=.5, keep_n=10000)

In [10]:
# Replace dataset with numeric ids words in vocab (and exclude all other words)
corpus = [dictionary.doc2bow(text) for text in data]

In [21]:
num_topics=50

Now let's run a topic model on this data using gensim's built-in LDA.

In [22]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           passes=10,
                                           alpha='auto')

We can get a sense of what the topics are by printing the top 10 words with highest $P(word \mid topic)$ for each topic

In [23]:
for i in range(num_topics):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model.show_topic(i, topn=10)])))

topic 0:	ghost spirit kane ghosts gray portrait dr. matthews eyes believe
topic 1:	toys toy buzz machine jensen jerusalem claw atomic mr. lunar
topic 2:	miller cooper brown woo butch pope park fraternity indian ford
topic 3:	kill killed police car kills gun house killing shoots killer
topic 4:	team escape kill dr. group world find killed time reveals
topic 5:	tells day night n't goes school asks party house leaves
topic 6:	band rock concert house music dj singer baby mother thorn
topic 7:	bush nash schmidt doom iraq w. henderson mass oil whip
topic 8:	shaw infected virus zoo animals finch frost griffin nixon fowler
topic 9:	archer asteroid mick crocodile crow cartel wax hoffman crawl ace
topic 10:	team game school play win high coach football father college
topic 11:	book company chip store control plant work tuck business job
topic 12:	boys ring fight brothers dodge boxing hook lost hall pan
topic 13:	earth alien ship space planet aliens power humans crew mission
topic 14:	town find m

Another way of understanding topics is to print out the documents that have the highest topic representation -- i.e., for a given topic $k$, the documents with highest $P(topic=k | document)$.  How much do the documents listed here align with your understanding of the topics?

In [24]:
topic_model=lda_model 

topic_docs=[]
for i in range(num_topics):
    topic_docs.append({})
for doc_id in range(len(corpus)):
    doc_topics=topic_model.get_document_topics(corpus[doc_id])
    for topic_num, topic_prob in doc_topics:
        topic_docs[topic_num][doc_id]=topic_prob

for i in range(num_topics):
    print("%s\n" % ' '.join([term for term, freq in topic_model.show_topic(i, topn=10)]))
    sorted_x = sorted(topic_docs[i].items(), key=operator.itemgetter(1), reverse=True)
    for k, v in sorted_x[:5]:
        print("%s\t%.3f\t%s" % (i,v,doc_names[k]))
    print()
    
    

ghost spirit kane ghosts gray portrait dr. matthews eyes believe

0	0.266	Dorian Gray
0	0.130	Thir13en Ghosts
0	0.115	Hello Ghost
0	0.112	Ghost Dog: The Way of the Samurai
0	0.111	High Noon

toys toy buzz machine jensen jerusalem claw atomic mr. lunar

1	0.292	Toy Story
1	0.219	Toy Story 2
1	0.211	Toy Story 3
1	0.130	Toys
1	0.122	The Expendables

miller cooper brown woo butch pope park fraternity indian ford

2	0.890	Made in Dagenham
2	0.610	My Life In Ruins
2	0.579	The Man with One Red Shoe
2	0.510	Daisy
2	0.459	New In Town

kill killed police car kills gun house killing shoots killer

3	0.778	I Saw the Devil
3	0.764	Fatal Beauty
3	0.762	Smokin' Aces
3	0.753	Death Wish 4: The Crackdown
3	0.744	Gone

team escape kill dr. group world find killed time reveals

4	0.780	Terminator Salvation
4	0.761	Star Wars Episode II: Attack of the Clones
4	0.737	G.I. Joe: The Rise of Cobra
4	0.729	Wonder Woman
4	0.689	Tron

tells day night n't goes school asks party house leaves

5	0.954	Prom
5	0.941	Ja